## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

In [10]:
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

TypeError: tuple indices must be integers or slices, not str

In [17]:
for day1 in states:
    result=None
    day1Prob='{}*({})'.format(start_probability[day1],emission_probatility[day1]['dry'])
    result=start_probability[day1]*emission_probatility[day1]['dry']
    for day2 in states:
        day2Prob='{}*({})'.format(transition_probability[day1][day2],emission_probatility[day2]['dry'])
        result=result*transition_probability[day1][day2]*emission_probatility[day2]['dry']
        for day3 in states:
            day3Prob='{}*({})'.format(transition_probability[day2][day3],emission_probatility[day3]['dry'])
            result=result*transition_probability[day2][day3]*emission_probatility[day3]['dry']
            print('{}, {}, {}: {}*{}*{} = {:.6f}'.format(day1.capitalize(),day2.capitalize(),day3.capitalize(),day1Prob,day2Prob,day3Prob,result))

Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
Sunny, Sunny, Rainy: 0.4*(0.6)*0.6*(0.6)*0.4*(0.1) = 0.001244
Sunny, Rainy, Sunny: 0.4*(0.6)*0.4*(0.1)*0.3*(0.6) = 0.000009
Sunny, Rainy, Rainy: 0.4*(0.6)*0.4*(0.1)*0.7*(0.1) = 0.000001
Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888
Rainy, Sunny, Rainy: 0.6*(0.1)*0.3*(0.6)*0.4*(0.1) = 0.000156
Rainy, Rainy, Sunny: 0.6*(0.1)*0.7*(0.1)*0.3*(0.6) = 0.000002
Rainy, Rainy, Rainy: 0.6*(0.1)*0.7*(0.1)*0.7*(0.1) = 0.000000


```
觀察狀態 = 'dry', 'dry', 'dry'
Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888
###<your answers>###

最大機率為: Sunny, Sunny, Sunny
```

### 根據上述條件, 寫出Viterbi應用程式

In [21]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

In [22]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        V[0][y] = start_p[y] * emit_p[y][obs[0]]
        path[y] = [y]

    # Run Viterbi for t > 0
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}

        for cur_state in states:
            (prob, state) = max([(V[t-1][pre_state] * trans_p[pre_state][cur_state] * emit_p[cur_state][obs[t]], pre_state) for pre_state in states])
            V[t][cur_state] = prob
            newpath[cur_state] = path[state] + [cur_state]

        # Don't need to remember the old paths
        path = newpath
    
    (prob, state) = max([(V[len(obs) - 1][final_state], final_state) for final_state in states])
    return (prob, path[state])

In [23]:
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probatility)

In [24]:
result

(0.031103999999999993, ['sunny', 'sunny', 'sunny'])